<a href="https://colab.research.google.com/github/kojikokojiko/portfoilio/blob/master/M5/val%E4%BA%88%E6%B8%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
%cd /content/drive/My Drive/kaggle/M5/iwase

/content/drive/.shortcut-targets-by-id/18e73-MFsSq-bfNImjjh-axhWYXPQE-7c/kaggle/M5/iwase


In [0]:
!ls


 c_weight.pickle		   'val_lags features '        val訓練WI
 カラム.gdoc			   'val_pickle1 '	      'val_調整 '
'lags features'			   'val_pickle2 '	       予測ノーマル
 pickle1.ipynb			   'val_pickle3 '	       予測CA　いわ
 pickle2.ipynb			    val_shop_id分け+mean_enc   予測.ipynb
 pickle3.ipynb			   'val予測 '		      '予測TX ryushi'
 sample_submission変換		    val訓練CA		      '予測WI takemi'
 shop_id分け+mean_enc上書き.ipynb   val訓練TX		       調整.ipynb


In [0]:
import pandas as pd
import numpy as np
import os, sys, gc, time, warnings, pickle, psutil, random

# VER変更


In [0]:
VER=4

In [0]:
STORE_IDS=[
 'CA_1',
 'CA_2',
 'CA_3',
 'CA_4',
 'TX_1',
 'TX_2',
 'TX_3',
 'WI_1',
 'WI_2',
 'WI_3'
]

In [0]:
all_preds=pd.DataFrame()


In [0]:
def get_base_test():
    base_test = pd.DataFrame()

    for store in STORE_IDS:
        temp_df = pd.read_pickle('../testpkl/val/ver'+str(VER)+'/val_test_v'+str(VER)+store+'.pkl')
        temp_df['store_id'] = store
        base_test = pd.concat([base_test, temp_df]).reset_index(drop=True)
    
    return base_test

In [0]:
base_test = get_base_test()

In [0]:
base_test

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,release,wm_yr_wk,sell_price,price_max,price_min,price_std,price_mean,price_norm,price_nunique,item_nunique,price_momentum,price_momentum_m,price_momentum_y,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,tm_d,tm_w,tm_m,tm_y,tm_wm,tm_dw,tm_w_end,enc_state_id_mean,enc_state_id_std,enc_store_id_mean,enc_store_id_std,enc_cat_id_mean,...,enc_dept_id_std,enc_state_id_cat_id_mean,enc_state_id_cat_id_std,enc_state_id_dept_id_mean,enc_state_id_dept_id_std,enc_store_id_cat_id_mean,enc_store_id_cat_id_std,enc_store_id_dept_id_mean,enc_store_id_dept_id_std,enc_item_id_mean,enc_item_id_std,enc_item_id_state_id_mean,enc_item_id_state_id_std,enc_item_id_store_id_mean,enc_item_id_store_id_std,sales_lag_28,sales_lag_29,sales_lag_30,sales_lag_31,sales_lag_32,sales_lag_33,sales_lag_34,sales_lag_35,sales_lag_36,sales_lag_37,sales_lag_38,sales_lag_39,sales_lag_40,sales_lag_41,sales_lag_42,rolling_mean_28_7,rolling_std_28_7,rolling_mean_28_14,rolling_std_28_14,rolling_mean_28_28,rolling_std_28_28,rolling_mean_28_56,rolling_std_28_56,rolling_mean_28_112,rolling_std_28_112
0,HOBBIES_1_001_CA_1_validation,1437,3,1,CA_1,CA,1814,0.0,224,11551,8.257812,9.578125,8.257812,0.152100,8.289062,0.862305,3.0,5,1.0,1.000000,0.993652,30,4,4,2,0,0,0,16,2,1,5,3,5,1,1.636719,4.460938,1.636719,4.460938,1.003906,...,3.541016,1.003906,3.121094,1.260742,3.541016,1.003906,3.121094,1.260742,3.541016,0.589844,0.849121,0.587891,1.048828,0.589844,0.849121,1.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.714355,1.112305,0.643066,9.287109e-01,0.571289,0.835938,0.571289,0.782715,0.589355,0.789062
1,HOBBIES_1_002_CA_1_validation,1438,3,1,CA_1,CA,1814,0.0,20,11551,3.970703,3.970703,3.970703,0.000000,3.970703,1.000000,1.0,131,1.0,1.000000,1.000000,30,4,4,2,0,0,0,16,2,1,5,3,5,1,1.636719,4.460938,1.636719,4.460938,1.003906,...,3.541016,1.003906,3.121094,1.260742,3.541016,1.003906,3.121094,1.260742,3.541016,0.278076,0.586914,NaN,NaN,0.278076,0.586914,5.0,1.0,1.0,2.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,0.0,0.0,1.0,2.0,1.428711,1.717773,1.071289,1.328125e+00,0.643066,1.095703,0.571289,0.911621,0.428467,0.791016
2,HOBBIES_1_003_CA_1_validation,1439,3,1,CA_1,CA,1814,0.0,300,11551,2.970703,2.970703,2.970703,0.000000,2.970703,1.000000,1.0,118,1.0,1.000000,1.000000,30,4,4,2,0,0,0,16,2,1,5,3,5,1,1.636719,4.460938,1.636719,4.460938,1.003906,...,3.541016,1.003906,3.121094,1.260742,3.541016,1.003906,3.121094,1.260742,3.541016,0.353760,0.696289,NaN,NaN,0.353760,0.696289,5.0,1.0,0.0,2.0,2.0,1.0,2.0,3.0,4.0,1.0,1.0,3.0,2.0,0.0,0.0,1.857422,1.573242,1.928711,1.439453e+00,1.571289,1.344727,1.017578,1.228516,0.785645,1.051758
3,HOBBIES_1_004_CA_1_validation,1440,3,1,CA_1,CA,1814,3.0,5,11551,4.640625,4.640625,4.339844,0.145264,4.527344,1.000000,2.0,2,1.0,1.024414,1.000000,30,4,4,2,0,0,0,16,2,1,5,3,5,1,1.636719,4.460938,1.636719,4.460938,1.003906,...,3.541016,1.003906,3.121094,1.260742,3.541016,1.003906,3.121094,1.260742,3.541016,1.750977,1.994141,1.069336,1.847656,1.750977,1.994141,0.0,2.0,1.0,2.0,3.0,2.0,2.0,2.0,0.0,1.0,1.0,0.0,0.0,2.0,3.0,1.713867,0.951172,1.286133,9.946289e-01,1.892578,1.286133,1.678711,1.335938,1.786133,1.933594
4,HOBBIES_1_005_CA_1_validation,1441,3,1,CA_1,CA,1814,0.0,16,11551,2.880859,3.080078,2.480469,0.150146,2.941406,0.935059,4.0,161,1.0,0.992188,1.000000,30,4,4,2,0,0,0,16,2,1,5,3,5,1,1.636719,4.460938,1.636719,4.460938,1.003906,...,3.541016,1.003906,3.121094,1.260742,3.541016,1.003906,3.121094,1.260742,3.541016,1.026367,1.310547,NaN,NaN,1.026367,1.310547,3.0,5.0,1.0,3.0,0.0,3.0,3.0,2.0,1.0,2.0,1.0,2.0,1.0,0.0,2.0,2.572266,1.618164,1.928711,1.384766e+00,1.357422,1.283203,1.357422,1.227539,1.330078,1.218750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [0]:
# def df_parallelize_run(func, t_split):
#     num_cores = np.min([N_CORES,len(t_split)])
#     pool = Pool(num_cores)
#     df = pd.concat(pool.map(func, t_split), axis=1)
#     pool.close()
#     pool.join()
#     return df

In [0]:
# def make_lag(LAG_DAY):
#     lag_df = base_test[['id','d',TARGET]]
#     col_name = 'sales_lag_'+str(LAG_DAY)
#     lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(LAG_DAY)).astype(np.float16)
#     return lag_df[[col_name]]


# def make_lag_roll(LAG_DAY):
#     shift_day = LAG_DAY[0]
#     roll_wind = LAG_DAY[1]
#     lag_df = base_test[['id','d',TARGET]]
#     col_name = 'rolling_mean_tmp_'+str(shift_day)+'_'+str(roll_wind)
#     lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(shift_day).rolling(roll_wind).mean())
#     return lag_df[[col_name]]

In [0]:
#LIMITS and const
TARGET      = 'sales'            # Our target
START_TRAIN = 0                  # We can skip some rows (Nans/faster training)
END_TRAIN   = 1913              # End day of our train set
P_HORIZON   = 28                 # Prediction horizon

In [0]:

remove_features = [
                  #  デフォルト
                   'id',
                   'state_id',
                   'store_id',
                   'date',
                   'wm_yr_wk',
                   'd',
                   TARGET,

                  #  随時追加
                   'sales_lag_tmp_1',
                   'sales_lag_tmp_7',
                   




                   

                   ]

In [0]:
TMP_LAG_DAYS = [col for col in [1,7,14]]

In [0]:
TMP_ROLS_SPLIT = []
for i in [1,7,14]:
    for j in [7,14,28,56]:
        TMP_ROLS_SPLIT.append([i,j])

In [0]:
main_time = time.time()
base_test = get_base_test()
for PREDICT_DAY in range(1,29):
  print('Predict | Day:', PREDICT_DAY)
  start_time = time.time()
  
  grid_df=base_test.copy()

  for LAG_DAY in TMP_LAG_DAYS:
    col_name='sales_lag_tmp_'+str(LAG_DAY)
    grid_df[col_name]=grid_df.groupby(['id'])[TARGET].transform(lambda x:x.shift(LAG_DAY)).astype(np.float16)
  print("GET_LAGS")  

  for i in TMP_ROLS_SPLIT:
    shift_day=i[0]
    roll_wind=i[1]
    col_name='rolling_mean_tmp_'+str(shift_day)+'_'+str(roll_wind)
    grid_df[col_name]=grid_df.groupby(['id'])[TARGET].transform(lambda x:x.shift(shift_day).rolling(roll_wind).mean())




  MODEL_FEATURES=[col for col in list(grid_df) if col not in remove_features]

  for store in STORE_IDS:
     model_path = '../model/val/ver'+str(VER)+'/val_lgb_model_v'+str(VER)+'_'+store+'.bin'


     estimator = pickle.load(open(model_path, 'rb'))

     days_mask=base_test['d']==(END_TRAIN+PREDICT_DAY)
     store_mask=base_test['store_id']==store

     mask=(days_mask)&(store_mask)

     base_test[TARGET][mask] = estimator.predict(grid_df[mask][MODEL_FEATURES])

    



    
  temp_df=base_test[days_mask][['id',TARGET]]
  temp_df.columns = ['id','F'+str(PREDICT_DAY)]
  if 'id' in list(all_preds):
      all_preds = all_preds.merge(temp_df, on=['id'], how='left')
  else:
      all_preds = temp_df.copy()

  print('#'*10, ' %0.2f min round |' % ((time.time() - start_time) / 60),
                  ' %0.2f min total |' % ((time.time() - main_time) / 60),
                  ' %0.2f day sales |' % (temp_df['F'+str(PREDICT_DAY)].sum()))
  del temp_df
    
all_preds = all_preds.reset_index(drop=True)
all_preds



    













Predict | Day: 1
GET_LAGS


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


##########  6.72 min round |  6.85 min total |  36992.00 day sales |
Predict | Day: 2
GET_LAGS
##########  6.54 min round |  13.39 min total |  35072.00 day sales |
Predict | Day: 3
GET_LAGS
##########  6.57 min round |  19.96 min total |  34496.00 day sales |
Predict | Day: 4
GET_LAGS
##########  6.58 min round |  26.54 min total |  34976.00 day sales |
Predict | Day: 5
GET_LAGS
##########  6.55 min round |  33.09 min total |  41344.00 day sales |
Predict | Day: 6
GET_LAGS
##########  6.51 min round |  39.60 min total |  50688.00 day sales |
Predict | Day: 7
GET_LAGS
##########  6.54 min round |  46.13 min total |  52896.00 day sales |
Predict | Day: 8
GET_LAGS
##########  6.58 min round |  52.71 min total |  43776.00 day sales |
Predict | Day: 9
GET_LAGS
##########  6.66 min round |  59.37 min total |  44128.00 day sales |
Predict | Day: 10
GET_LAGS
##########  6.64 min round |  66.01 min total |  38560.00 day sales |
Predict | Day: 11
GET_LAGS
##########  6.52 min round |  72.52 min

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.788086,0.711426,0.681152,0.706543,0.812012,1.020508,1.094727,0.828613,0.836914,0.736816,0.718750,0.829102,0.974121,0.773438,0.760742,0.705078,0.725098,0.641113,0.794434,1.022461,0.963867,0.798828,0.716309,0.660156,0.683594,0.818848,1.006836,0.963379
1,HOBBIES_1_002_CA_1_validation,0.170898,0.155884,0.145874,0.161011,0.207520,0.305420,0.316650,0.193481,0.186523,0.200562,0.201294,0.223633,0.302979,0.245850,0.197998,0.199585,0.187988,0.189575,0.228638,0.290771,0.303955,0.215088,0.209961,0.225098,0.230957,0.268555,0.338135,0.361084
2,HOBBIES_1_003_CA_1_validation,0.458740,0.449707,0.462158,0.480713,0.619629,0.754883,0.701172,0.464844,0.466064,0.435547,0.447998,0.547363,0.697754,0.510742,0.453857,0.476562,0.468506,0.481201,0.575195,0.701660,0.727051,0.468994,0.448486,0.448730,0.482666,0.567871,0.723633,0.697266
3,HOBBIES_1_004_CA_1_validation,1.728516,1.380859,1.362305,1.541992,1.938477,2.871094,3.185547,1.608398,1.454102,1.387695,1.682617,1.771484,2.888672,2.419922,1.430664,1.410156,1.351562,1.353516,1.919922,2.470703,3.300781,1.607422,1.548828,1.307617,1.394531,1.892578,3.085938,3.535156
4,HOBBIES_1_005_CA_1_validation,0.924805,0.879883,0.856445,0.818359,1.138672,1.582031,1.447266,1.019531,0.997070,0.970703,0.930176,1.034180,1.445312,1.044922,0.934082,0.943359,0.937012,0.996094,1.131836,1.520508,1.535156,0.942871,0.841797,0.884766,0.892090,1.151367,1.497070,1.439453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,0.325439,0.300293,0.298096,0.303467,0.361816,0.434570,0.417480,0.454346,0.477783,0.375977,0.452148,0.414307,0.495850,0.517578,0.500488,0.414551,0.471191,0.462158,0.419189,0.509277,0.560547,0.487549,0.376709,0.368896,0.321777,0.355957,0.379150,0.449219
30486,FOODS_3_824_WI_3_validation,0.266846,0.238159,0.205566,0.196411,0.239014,0.294922,0.294922,0.340820,0.348633,0.264404,0.267090,0.303955,0.311523,0.303223,0.361328,0.269287,0.345703,0.318604,0.273193,0.323730,0.406006,0.274902,0.218506,0.221802,0.201538,0.218506,0.282959,0.299805
30487,FOODS_3_825_WI_3_validation,0.682129,0.537598,0.461914,0.457520,0.634277,0.672852,0.788574,1.154297,1.146484,0.826172,1.085938,1.150391,1.150391,1.171875,1.349609,0.906250,1.211914,1.204102,0.891113,1.248047,1.370117,0.956543,0.716797,0.671387,0.598145,0.633789,0.818848,0.750000
30488,FOODS_3_826_WI_3_validation,1.019531,0.979492,0.864746,0.806641,0.997070,1.308594,1.152344,1.189453,1.208984,0.992676,1.158203,1.083008,1.287109,1.285156,1.206055,1.028320,1.058594,0.965332,1.060547,1.355469,1.416992,1.081055,0.993652,0.889160,0.901367,1.052734,1.226562,1.292969


In [0]:
all_preds

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.788086,0.711426,0.681152,0.706543,0.812012,1.020508,1.094727,0.828613,0.836914,0.736816,0.718750,0.829102,0.974121,0.773438,0.760742,0.705078,0.725098,0.641113,0.794434,1.022461,0.963867,0.798828,0.716309,0.660156,0.683594,0.818848,1.006836,0.963379
1,HOBBIES_1_002_CA_1_validation,0.170898,0.155884,0.145874,0.161011,0.207520,0.305420,0.316650,0.193481,0.186523,0.200562,0.201294,0.223633,0.302979,0.245850,0.197998,0.199585,0.187988,0.189575,0.228638,0.290771,0.303955,0.215088,0.209961,0.225098,0.230957,0.268555,0.338135,0.361084
2,HOBBIES_1_003_CA_1_validation,0.458740,0.449707,0.462158,0.480713,0.619629,0.754883,0.701172,0.464844,0.466064,0.435547,0.447998,0.547363,0.697754,0.510742,0.453857,0.476562,0.468506,0.481201,0.575195,0.701660,0.727051,0.468994,0.448486,0.448730,0.482666,0.567871,0.723633,0.697266
3,HOBBIES_1_004_CA_1_validation,1.728516,1.380859,1.362305,1.541992,1.938477,2.871094,3.185547,1.608398,1.454102,1.387695,1.682617,1.771484,2.888672,2.419922,1.430664,1.410156,1.351562,1.353516,1.919922,2.470703,3.300781,1.607422,1.548828,1.307617,1.394531,1.892578,3.085938,3.535156
4,HOBBIES_1_005_CA_1_validation,0.924805,0.879883,0.856445,0.818359,1.138672,1.582031,1.447266,1.019531,0.997070,0.970703,0.930176,1.034180,1.445312,1.044922,0.934082,0.943359,0.937012,0.996094,1.131836,1.520508,1.535156,0.942871,0.841797,0.884766,0.892090,1.151367,1.497070,1.439453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,0.325439,0.300293,0.298096,0.303467,0.361816,0.434570,0.417480,0.454346,0.477783,0.375977,0.452148,0.414307,0.495850,0.517578,0.500488,0.414551,0.471191,0.462158,0.419189,0.509277,0.560547,0.487549,0.376709,0.368896,0.321777,0.355957,0.379150,0.449219
30486,FOODS_3_824_WI_3_validation,0.266846,0.238159,0.205566,0.196411,0.239014,0.294922,0.294922,0.340820,0.348633,0.264404,0.267090,0.303955,0.311523,0.303223,0.361328,0.269287,0.345703,0.318604,0.273193,0.323730,0.406006,0.274902,0.218506,0.221802,0.201538,0.218506,0.282959,0.299805
30487,FOODS_3_825_WI_3_validation,0.682129,0.537598,0.461914,0.457520,0.634277,0.672852,0.788574,1.154297,1.146484,0.826172,1.085938,1.150391,1.150391,1.171875,1.349609,0.906250,1.211914,1.204102,0.891113,1.248047,1.370117,0.956543,0.716797,0.671387,0.598145,0.633789,0.818848,0.750000
30488,FOODS_3_826_WI_3_validation,1.019531,0.979492,0.864746,0.806641,0.997070,1.308594,1.152344,1.189453,1.208984,0.992676,1.158203,1.083008,1.287109,1.285156,1.206055,1.028320,1.058594,0.965332,1.060547,1.355469,1.416992,1.081055,0.993652,0.889160,0.901367,1.052734,1.226562,1.292969


In [0]:
submission = pd.read_csv('../input/sample_submission.csv')[['id']]

In [0]:
# submission = pd.read_csv('../input/sample_submission.csv')[['id']]
submission = submission.merge(all_preds, on=['id'], how='left').fillna(0)
submission.to_csv('../submission/val/val_submission_v'+str(VER)+'.csv', index=False)

In [0]:
submission


,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.788086,0.711426,0.681152,0.706543,0.812012,1.020508,1.094727,0.828613,0.836914,0.736816,0.718750,0.829102,0.974121,0.773438,0.760742,0.705078,0.725098,0.641113,0.794434,1.022461,0.963867,0.798828,0.716309,0.660156,0.683594,0.818848,1.006836,0.963379
1,HOBBIES_1_002_CA_1_validation,0.170898,0.155884,0.145874,0.161011,0.207520,0.305420,0.316650,0.193481,0.186523,0.200562,0.201294,0.223633,0.302979,0.245850,0.197998,0.199585,0.187988,0.189575,0.228638,0.290771,0.303955,0.215088,0.209961,0.225098,0.230957,0.268555,0.338135,0.361084
2,HOBBIES_1_003_CA_1_validation,0.458740,0.449707,0.462158,0.480713,0.619629,0.754883,0.701172,0.464844,0.466064,0.435547,0.447998,0.547363,0.697754,0.510742,0.453857,0.476562,0.468506,0.481201,0.575195,0.701660,0.727051,0.468994,0.448486,0.448730,0.482666,0.567871,0.723633,0.697266
3,HOBBIES_1_004_CA_1_validation,1.728516,1.380859,1.362305,1.541992,1.938477,2.871094,3.185547,1.608398,1.454102,1.387695,1.682617,1.771484,2.888672,2.419922,1.430664,1.410156,1.351562,1.353516,1.919922,2.470703,3.300781,1.607422,1.548828,1.307617,1.394531,1.892578,3.085938,3.535156
4,HOBBIES_1_005_CA_1_validation,0.924805,0.879883,0.856445,0.818359,1.138672,1.582031,1.447266,1.019531,0.997070,0.970703,0.930176,1.034180,1.445312,1.044922,0.934082,0.943359,0.937012,0.996094,1.131836,1.520508,1.535156,0.942871,0.841797,0.884766,0.892090,1.151367,1.497070,1.439453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
60976,FOODS_3_824_WI_3_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
60977,FOODS_3_825_WI_3_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
60978,FOODS_3_826_WI_3_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [0]:
all_preds_X=pd.concat([val_sub,all_preds])

NameError: ignored

In [0]:
all_preds_X

In [0]:
submission


In [0]:
submission

# ここまでで

In [0]:
# val_sub=pd.read_pickle('../input/val_submission.csv')

In [0]:
# submission=pd.merge(submission,val_sub,on=['id'],how='left')

In [0]:
# submission